### IAPPT
Este archivo contiene la creacion y entrenamiento del modelo que usamos para determinar si la mano esta en posicion de piedra, papel o tijera.
Antes de nada ejecutaremos los imports necesarios.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from IPython.display import clear_output
import pandas as pd


### Datasets
Primero hemos de entender que estamos pasando a nuestro modelo exactamente.
Mediante mediapie obtenemos una serie de puntos llamados landmarks, cada uno con un numero representandolo. 
De recolectar estos datos y guardarlos en el csv se encarga el ejecutable ```csv_generator```.
![alt text](https://google.github.io/mediapipe/images/mobile/hand_landmarks.png)

De todas estos puntos solo nos interesan 9 puntos el 6,8,10,12,14,16,18,20.
Y quedan definidos asi en el csv:

- gesto: que se representa si esa fila que gesto de PPT representa
- muneca_Y: coordenada Y del landmark 0
- muneca_X: coordenada X del landmark 0
- indicep_Y: coordenada Y del landmark 6
- indicep_X: coordenada X del landmark 6
- indicet_Y: coordenada Y del landmark 8
- indicet_Y: coordenada X del landmark 8
- ...

In [48]:
# A almacenamos los datos de los ficheros
data_train = pd.read_csv('training_data.csv')
data_eval = pd.read_csv('eval_data.csv')

Antes de usar el dataset eliminamos la columna que representa el resultado y la almacenamos en una variable



In [49]:
# Eliminamos el campo con el resultado
train_y = data_train.pop('gesto')
eval_y = data_eval.pop('gesto') 

In [50]:
# Especificamos los campos
feature_columns = []
for key in data_train.keys():
  feature_columns.append(tf.feature_column.numeric_column(key=key))

### Input funtion
Esta es la funcion que usaremos para alimentar a nuestro modelo con los datos, sus parametros

- ```features``` lista de los landmarks de la mano.

- ```labels``` columna del csv que contiene el gesto que representan los landmarks de features.

- ```training``` en caso de ser True el la funcion devuelve filas seleccionadas de manera aleatoria del dataset.

- ```batch_size``` cantidad de filas que se retunean a la vez al modelo.

In [51]:
def input_function(features, labels, training=True, batch_size=128):
  # Convert the inputs to a Dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  # Mezclar los datos
  if training:
      dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

### Creando el modelo
Para este caso lo que necesitamos el un modelo que sepa clasificar los datos en 3 categorias correspondiendo a PPT.
Tenemos varias opciones pero yo he optado por ```DNNClassifier```.

Los parametros son estos
- ```feature_columns``` lista de los landmarks de la mano en total 18.

- ```hidden_units``` son las dos capas ocultas con 30 y 10 nodos respectivamente.

- ```n_classes``` el numeros de grupos en el que ha de agrupar el resultado.

- ```model_dir``` donde queda almacenado el modelo.

In [73]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[30, 10],
    n_classes=3,
    model_dir='model'
)
clear_output(False)

### Entrenar el modelo
Aqui es donde empieza a aprender y clasificar
En nuestro caso le hemos especificado los ```steps```, es decir que recibira x batches de 128 filas de datos.

In [84]:
classifier.train(input_fn=lambda: input_function(data_train, train_y, training=True),steps=7000)
clear_output(False)

### Resultados

In [ ]:
#prueba
classifier.predict(input_function())

In [ ]:
# Visualizar la precision del modelo (de 0.0 a 1.0)
eval_result = classifier.evaluate(input_fn=lambda: input_function(data_eval, eval_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}'.format(**eval_result))
